# Thoth 0.5.0 - Example 3 Guided Notebook - Scoring

This notebook is a supportive material for example 3 of Thoth's 0.5.0 release. You can follow instructions in [Thamos repository with examples](https://github.com/thoth-station/thamos/tree/master/examples/scoring) if you would like to use Thamos CLI and communicate with a deployed Thoth backend.

See internal document for more info and clarification in [Google Docs](https://docs.google.com/document/d/1QflQpGXtOuHFFC2hkEFBlu0JmCQWEnXdgryvwxCNXpQ/edit#) - section "Example 3".

## Initial graph database setup

In order to go through this scenario, first we need to connect to a graph database instance. This notebook is playeble from within your computer, it inserts all the data into a provided JanusGraph instance, so select a graph database instance you would like to use. If you want to run this script purely on your local machine, setup your local graph database as [described in the README file of thoth-station/janusgraph-thoth-config repo](https://github.com/thoth-station/janusgraph-thoth-config#running-janusgraph-instance-locally). Ideally, just clone the repo and issue the following command to setup your local JanusGraph database instance:

```
sudo ./local.sh all
```

In [1]:
# Configure JanusGraph instance to talk to:
JANUSGRAPH_SERVICE_HOST = 'localhost'

# For directly talking to test environment, uncomment the following line:
# JANUSGRAPH_SERVICE_HOST = 'janusgraph.test.thoth-station.ninja'

Now let's connect to desired JanusGraph database and check if we are properly connected:

In [2]:
from thoth.storages import GraphDatabase

# Instantiate and connect the JanusGraph database.
graph = GraphDatabase.create(JANUSGRAPH_SERVICE_HOST)
graph.connect()

graph.is_connected()

True

In the next step we download result of a [thoth-solver](https://github.com/thoth-station/solver) run which resolved all the required stacks (Flask, PyYAML and all their transitive dependencies) to this date.

In [3]:
import requests
from thoth.common import timestamp2datetime

SOLVER_DOCUMENT_URL = 'https://raw.githubusercontent.com/thoth-station/misc/master/examples/scoring/resolved.json'

response = requests.get(SOLVER_DOCUMENT_URL)
response.raise_for_status()
solver_document = response.json()

print("Document covers all transitive packages which can be installed when %r is installed (no version specifier, any version)." % solver_document["metadata"]["arguments"]["pypi"]["requirements"])
print("Stacks were resolved at", timestamp2datetime(solver_document["metadata"]["timestamp"]))

Document covers all transitive packages which can be installed when 'flask\\npyyaml\\n' is installed (no version specifier, any version).
Stacks were resolved at 2019-03-12 12:42:04+00:00


This document states packages and resolved dependencies. These data are used to construct entries inside graph database which are used by Thoth's resolver to construct software stacks. Note the construction of stacks is done offline on high frequencies (installation of Python packages is slow). The solver document captures environment details for which solver was run (see other Thoth notebooks for clarification why is that happening).

In [4]:
from thoth.storages import SolverResultsStore

document_id = SolverResultsStore.get_document_id(solver_document)
solver_name = SolverResultsStore.get_solver_name_from_document_id(document_id)
graph.parse_python_solver_name(solver_name)

{'os_name': 'fedora', 'os_version': '29', 'python_version': '3.6'}

Let's sync the solver document into graph so we can later on use these dependency graphs:

In [5]:
%%time

graph.sync_solver_result(solver_document)

CPU times: user 17.1 s, sys: 1 s, total: 18.1 s
Wall time: 44.2 s


## Stack generation and scoring

In this noteboook we will show scoring of a software stack and how Thoth finds an optimal stack for you.

Imagine a stack made out of two libraries "`simplelib`" and "`anotherlib`". They together influence how software which uses them works. On the figure below, there are shown scores a scoring function gave when there were used different versions of libraries mentioned above.

![alt text](https://raw.githubusercontent.com/thoth-station/misc/master/fig/score_3d_raw.png "Scoring function visualization")

As the image above is not easily readable (discrete values), let's interpolate results - this way we will see a surface the scoring function is creating considering different versions of `simplelib` and `anotherlib`. 

![alt text](https://raw.githubusercontent.com/thoth-station/misc/master/fig/score_3d.png "Scoring function visualization")

With the theoretical example above, let's demo this using Thoth. Our scoring function will be "how many CVEs are present in a software stack". Even though CVEs are "low hanging fruit", they can nicely show approaches Thoth is performing. In chapters below, we will show how this approach can be extended considering additional vector in the scoring function, such as performance characteristicts of a software stack. From now on, let's imagine `simplelib` is Flask and `anotherlib` is PyYAML. Let's assume other dependencies (the transitive ones of Flask and PyYAML) do not have any CVEs so we can project stacks into a 3D space as shown above (if there would be another library with a CVE, we would need to add a new dimension).

To demo the use case, let's create an application which uses Python libraries - [Flask](https://pypi.org/project/Flask/) and [PyYAML](https://pypi.org/project/PyYAML/). As Thoth has a resolver implemented on top of graph database, it can generate software stacks and score them on high frequencies (see for example [`libdependency_graph.so` library which Thoth uses under the hood](https://github.com/thoth-station/adviser/blob/master/docs/libdependency_graph.md)) and perform scoring based on observations.

In one of the cells above, we fed [thoth-solver](https://github.com/thoth-station/solver/) results into graph database so that we have a notion about all the transitive dependencies of Python packages our user's application uses. 

Next, let's insert some CVE related information.

Thoth, as of now, uses [pyup.io](https://pyup.io/)'s safety database which curates CVEs in Python ecosystem. The  sync of this database into Thoth's knowledge base is done by Thoth's [cve-update-job](https://github.com/thoth-station/cve-update-job) component in a deployment. As we work in a Jupyter Notebook, possibly talking to originally un-initialized graph database instance, let's manually insert some CVEs into our database:

In [6]:
# Download pyup.io safety database:
import requests

SAFETY_DB_URL = "https://raw.githubusercontent.com/pyupio/safety-db/master/data/insecure_full.json"

response = requests.get(SAFETY_DB_URL)
response.raise_for_status()
cve_database = response.json()

# We will use Thoth's resolver implemented on top of graph
# database to resolve packages which are affected by CVEs we are interest in to demo:
from thoth.python import PackageVersion
from thoth.adviser.python.solver import PythonPackageGraphSolver

# Instantiate Thoth's graph solver:
graph_solver = PythonPackageGraphSolver(graph_db=graph)

for cve in cve_database.get("flask", []):
    print("---> Syncing Flask CVE record into graph database, cve: %r (CVE id: %r)" % (cve["id"], cve["cve"]))
    print("CVE %r is affecting Flask versions %r" % (cve["id"], cve["v"]))
    
    # Resolve versions of affected Flask versions:
    versions = graph_solver.solve([PackageVersion(name="flask", version=cve["v"], develop=False)], all_versions=True)
    for package_version in versions['flask']:
        print("\tCreating record for affected Flask version: %r" % (package_version.locked_version))
        graph.create_python_cve_record(
            package_name=package_version.name,
            package_version=package_version.locked_version,
            index_url="https://pypi.org/simple",  # This is the index monitored by pyup.io safety db.
            record_id=cve["id"],
            version_range=cve["v"],
            advisory=cve["advisory"],
            cve=cve["cve"] or "N/A",
        )

2019-03-13 09:06:50,607 [20383] INFO     root:126: Logging to a Sentry instance is turned off
2019-03-13 09:06:50,609 [20383] INFO     root:148: Logging to rsyslog endpoint is turned off
2019-03-13 09:06:50,635 [20383] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'flask<0.12.3'


---> Syncing Flask CVE record into graph database, cve: 'pyup.io-36388' (CVE id: 'CVE-2018-1000656')
CVE 'pyup.io-36388' is affecting Flask versions '<0.12.3'
	Creating record for affected Flask version: '0.1'
	Creating record for affected Flask version: '0.2'
	Creating record for affected Flask version: '0.3'
	Creating record for affected Flask version: '0.3.1'
	Creating record for affected Flask version: '0.4'
	Creating record for affected Flask version: '0.5'
	Creating record for affected Flask version: '0.5.1'
	Creating record for affected Flask version: '0.5.2'
	Creating record for affected Flask version: '0.6'
	Creating record for affected Flask version: '0.6.1'
	Creating record for affected Flask version: '0.10'
	Creating record for affected Flask version: '0.10.1'
	Creating record for affected Flask version: '0.11'
	Creating record for affected Flask version: '0.11.1'
	Creating record for affected Flask version: '0.12'


2019-03-13 09:06:51,135 [20383] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'flask<0.6.1'


	Creating record for affected Flask version: '0.12.1'
	Creating record for affected Flask version: '0.12.2'
---> Syncing Flask CVE record into graph database, cve: 'pyup.io-25820' (CVE id: None)
CVE 'pyup.io-25820' is affecting Flask versions '<0.6.1'
	Creating record for affected Flask version: '0.1'
	Creating record for affected Flask version: '0.2'
	Creating record for affected Flask version: '0.3'
	Creating record for affected Flask version: '0.3.1'
	Creating record for affected Flask version: '0.4'
	Creating record for affected Flask version: '0.5'
	Creating record for affected Flask version: '0.5.1'
	Creating record for affected Flask version: '0.5.2'
	Creating record for affected Flask version: '0.6'


There is also a CVE with CVE ID [CVE-2017-18342](https://nvd.nist.gov/vuln/detail/CVE-2017-18342) which is affecting the most recent versions of PyYAML (version lower than 4.1 and, for our use case, higher than 3.05). The fix is present also in pre-releases of PyYaml and [the most recent version of PyYAML to this date 3.13](https://pypi.org/project/PyYAML/#history) is also vulnarable by this CVE.

The pyup.io does not state this CVE (too recent probably?), so let's insert it manually for now to the graph database:

In [7]:
print("---> Syncing PyYAML CVE record into graph database, cve: CVE-2017-18342")

# Resolve versions of affected PyYAML versions:
versions = graph_solver.solve([PackageVersion(name="pyyaml", version="<4.1,>3.05", develop=False)], all_versions=True)
for package_version in versions['pyyaml']:
    print("\tCreating record for affected PyYAML version: %r" % (package_version.locked_version))
    graph.create_python_cve_record(
        package_name=package_version.name,
        package_version=package_version.locked_version,
        index_url="https://pypi.org/simple",
        record_id="CVE-2017-18342",
        version_range="<4.1,>3.05",
        advisory="In PyYAML before 4.1, the yaml.load() API could execute arbitrary code. In other words, yaml.safe_load is not used.",
        cve="CVE-2017-18342",
    )

2019-03-13 09:06:51,377 [20383] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'pyyaml<4.1,>3.05'


---> Syncing PyYAML CVE record into graph database, cve: CVE-2017-18342
	Creating record for affected PyYAML version: '3.13rc1'
	Creating record for affected PyYAML version: '3.13b1'
	Creating record for affected PyYAML version: '3.10'
	Creating record for affected PyYAML version: '3.11'
	Creating record for affected PyYAML version: '3.12'
	Creating record for affected PyYAML version: '3.13'


We have fed all the data necessary for this notebook into our graph database instance. Let's have a look at user's stack by inspecting `Pipfile` and `Pipfile.lock`:

In [8]:
PIPFILE_URL = "https://raw.githubusercontent.com/thoth-station/thamos/master/examples/scoring/Pipfile"
PIPFILE_LOCK_URL = "https://raw.githubusercontent.com/thoth-station/thamos/master/examples/scoring/Pipfile.lock"

response = requests.get(PIPFILE_URL)
response.raise_for_status()
pipfile_str = response.text

response = requests.get(PIPFILE_LOCK_URL)
response.raise_for_status()

pipfile_lock_str = response.text

Direct dependencies which user directly uses in her/his application with configured index:

In [9]:
print(pipfile_str)

[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
flask = "*"
pyyaml = "*"




And the corresponding lockfile for user's stack.

In [10]:
print(pipfile_lock_str)

{
    "_meta": {
        "hash": {
            "sha256": "0fd39b0c8e7bc29f982d3de717ad9aa6c4f70d82b71ef2bee404f7c4a49487a8"
        },
        "pipfile-spec": 6,
        "requires": {},
        "sources": [
            {
                "name": "pypi",
                "url": "https://pypi.org/simple",
                "verify_ssl": true
            }
        ]
    },
    "default": {
        "click": {
            "hashes": [
                "sha256:2335065e6395b9e67ca716de5f7526736bfa6ceead690adf616d925bdc622b13",
                "sha256:5b94b49521f6456670fdb30cd82a4eca9412788a93fa6dd6df72c94d5a8ff2d7"
            ],
            "version": "==7.0"
        },
        "flask": {
            "hashes": [
                "sha256:9dc18a7c673bf0a6fada51e011fc411285a8301f6dfc1c000ebfa272b5e609e4"
            ],
            "index": "pypi",
            "version": "==0.6"
        },
        "itsdangerous": {
            "hashes": [
                "sha256:321b033d07f2a4136d3ec762eac9f16a10ccd60f

Thoth internally operates on "Project" abstraction, so let's instantiate one:

In [11]:
from thoth.python import Project

project = Project.from_strings(pipfile_str, pipfile_lock_str)
project.to_dict()

{'requirements': {'packages': {'flask': '*', 'pyyaml': '*'},
  'dev-packages': {},
  'source': [{'url': 'https://pypi.org/simple',
    'verify_ssl': True,
    'name': 'pypi'}]},
 'requirements_locked': {'_meta': {'sources': [{'url': 'https://pypi.org/simple',
     'verify_ssl': True,
     'name': 'pypi'}],
   'requires': {},
   'hash': {'sha256': '0fd39b0c8e7bc29f982d3de717ad9aa6c4f70d82b71ef2bee404f7c4a49487a8'},
   'pipfile-spec': 6},
  'default': {'click': {'version': '==7.0',
    'hashes': ['sha256:2335065e6395b9e67ca716de5f7526736bfa6ceead690adf616d925bdc622b13',
     'sha256:5b94b49521f6456670fdb30cd82a4eca9412788a93fa6dd6df72c94d5a8ff2d7']},
   'flask': {'version': '==0.6',
    'hashes': ['sha256:9dc18a7c673bf0a6fada51e011fc411285a8301f6dfc1c000ebfa272b5e609e4'],
    'index': 'pypi'},
   'itsdangerous': {'version': '==1.1.0',
    'hashes': ['sha256:321b033d07f2a4136d3ec762eac9f16a10ccd60f53c0c91af90217ace7ba1f19',
     'sha256:b12271b2047cb23eeb98c8b5622e2e5c5e9abd9784a153e9d8ef

Let's have a look what versions of PyYAML and Flask libraries are used:

In [12]:
print("Flask is used in version %r " % project.get_locked_package_version("flask").locked_version)

Flask is used in version '0.6' 


In [13]:
print("PyYAML is used in version %r " % project.get_locked_package_version("pyyaml").locked_version)

PyYAML is used in version '3.13' 


As you can see, CVEs known affect both versions of Flask and PyYAML. If you would take a closer look at stacks, evaluate version ranges of CVEs manually for the affected packages, you would come up with a solution to have a CVE-free software stack. The solution would be:

* Update Flask to version >0.12.2
* Downgrage PyYAML to lower version without CVE

Or:

* Update Flask to version >0.12.2
* Use a pre-release of PyYAML to get rid of PyYAML CVE vulnerability

As user did not configured pre-releases (`allow_prereleases` configuration option in Pipfile file, or run `pipenv install --pre` - see `Pipfile` at the end of this notebook as an example).

Now, we can let Thoth compute the best possible software stack. You can adjust configuration options for `Adviser`. Without any limitations, there are possibly 46,997,280 different stacks (upper bound, estimated based on number of packages and all the combinations they can create). Howerver, you can adjust parameters so that this number is lower (e.g. reducing number of recent versions for each package - `limit_latest_versions`).

Let's give adviser a try:

In [14]:
%%time
%env THOTH_ADVISER_SHOW_PACKAGES=1

from thoth.adviser.python import Adviser
from thoth.adviser.enums import RecommendationType

stack_info, advised_configuration, report = Adviser.compute_on_project(
    project,
    recommendation_type=RecommendationType.STABLE,
    count=5,                   # Number of best stacks reported in the output.
    limit=100,                # Limit number of stacks scored in total.
    limit_latest_versions=4,   # Consider only N latest versions of each package.
    dry_run=False,
    graph=graph,
)

2019-03-13 09:06:52,482 [20383] INFO     thoth.adviser.python.scoring:54: Using scoring function obtaining stable software stacks
2019-03-13 09:06:52,484 [20383] INFO     thoth.adviser.python.dependency_graph:440: Parsing and solving direct dependencies of the requested project
2019-03-13 09:06:52,485 [20383] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'flask'


env: THOTH_ADVISER_SHOW_PACKAGES=1


2019-03-13 09:06:52,573 [20383] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'pyyaml'
2019-03-13 09:06:52,652 [20383] INFO     thoth.adviser.python.dependency_graph:455: Retrieving transitive dependencies of direct dependencies
2019-03-13 09:06:54,395 [20383] INFO     thoth.adviser.python.dependency_graph:558: Sorting dependencies to preserve order of generated stacks
2019-03-13 09:06:56,191 [20383] INFO     thoth.adviser.python.dependency_graph:563: Cutting off unwanted dependencies
2019-03-13 09:06:56,527 [20383] WARNING  thoth.adviser.python.dependency_graph:573: Sorting dependencies to preserve order of generated stacks
2019-03-13 09:06:56,711 [20383] WARNING  thoth.adviser.python.dependency_graph:581: Removing older versions of packages, considering only 4 latest versions for each package
2019-03-13 09:06:56,741 [20383] INFO     thoth.adviser.python.dependency_graph:609: Creating dependency graph
2019-03-13 09:06:56,743 [20383] INFO     thoth.adviser.python.d

2019-03-13 09:06:57,031 [20383] INFO     thoth.adviser.python.scoring:226: Performance index for stack: nan
2019-03-13 09:06:57,058 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:06:57,060 [20383] DEBUG    thoth.adviser.python.bin.dependency_graph:195: Reached stack delimiter, yielding stack
2019-03-13 09:06:57,065 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a new stack, asking decision function for inclusion
2019-03-13 09:06:57,078 [20383] INFO     thoth.adviser.python.scoring:226: Performance index for stack: nan
2019-03-13 09:06:57,104 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:06:57,106 [20383] DEBUG    thoth.adviser.python.bin.dependency_graph:195: Reached stack delimiter, yielding stack
2019-03-13 09:06:57,110 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a 

2019-03-13 09:06:57,719 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a new stack, asking decision function for inclusion
2019-03-13 09:06:57,730 [20383] INFO     thoth.adviser.python.scoring:226: Performance index for stack: nan
2019-03-13 09:06:57,755 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:06:57,756 [20383] DEBUG    thoth.adviser.python.bin.dependency_graph:195: Reached stack delimiter, yielding stack
2019-03-13 09:06:57,757 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a new stack, asking decision function for inclusion
2019-03-13 09:06:57,769 [20383] INFO     thoth.adviser.python.scoring:226: Performance index for stack: nan
2019-03-13 09:06:57,793 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:06:57,794 [20383] DEBUG    thoth.adviser.python.bin.dependency_g

...

2019-03-13 09:06:59,078 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:06:59,079 [20383] DEBUG    thoth.adviser.python.bin.dependency_graph:195: Reached stack delimiter, yielding stack
2019-03-13 09:06:59,082 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a new stack, asking decision function for inclusion
2019-03-13 09:06:59,092 [20383] INFO     thoth.adviser.python.scoring:226: Performance index for stack: nan
2019-03-13 09:06:59,132 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:06:59,133 [20383] DEBUG    thoth.adviser.python.bin.dependency_graph:195: Reached stack delimiter, yielding stack
2019-03-13 09:06:59,135 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a new stack, asking decision function for inclusion
2019-03-13 09:06:59,150 [20383] INFO     thoth.adviser.pyt

2019-03-13 09:06:59,734 [20383] INFO     thoth.adviser.python.scoring:226: Performance index for stack: nan
2019-03-13 09:06:59,758 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:06:59,760 [20383] DEBUG    thoth.adviser.python.bin.dependency_graph:195: Reached stack delimiter, yielding stack
2019-03-13 09:06:59,761 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a new stack, asking decision function for inclusion
2019-03-13 09:06:59,773 [20383] INFO     thoth.adviser.python.scoring:226: Performance index for stack: nan
2019-03-13 09:06:59,796 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:06:59,798 [20383] DEBUG    thoth.adviser.python.bin.dependency_graph:195: Reached stack delimiter, yielding stack
2019-03-13 09:06:59,799 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a 

2019-03-13 09:07:00,358 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a new stack, asking decision function for inclusion
2019-03-13 09:07:00,371 [20383] INFO     thoth.adviser.python.scoring:226: Performance index for stack: nan
2019-03-13 09:07:00,397 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:07:00,399 [20383] DEBUG    thoth.adviser.python.bin.dependency_graph:195: Reached stack delimiter, yielding stack
2019-03-13 09:07:00,400 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a new stack, asking decision function for inclusion
2019-03-13 09:07:00,416 [20383] INFO     thoth.adviser.python.scoring:226: Performance index for stack: nan
2019-03-13 09:07:00,441 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:07:00,442 [20383] DEBUG    thoth.adviser.python.bin.dependency_g

2019-03-13 09:07:01,024 [20383] DEBUG    thoth.adviser.python.bin.dependency_graph:195: Reached stack delimiter, yielding stack
2019-03-13 09:07:01,026 [20383] INFO     thoth.adviser.python.dependency_graph:744: Found a new stack, asking decision function for inclusion
2019-03-13 09:07:01,036 [20383] INFO     thoth.adviser.python.scoring:226: Performance index for stack: nan
2019-03-13 09:07:01,060 [20383] INFO     thoth.adviser.python.dependency_graph:750: Decision function included the computed stack - result was 0.0
2019-03-13 09:07:01,061 [20383] INFO     thoth.adviser.python.advise:111: Reached graph traversal limit (100), stopping dependency graph traversal
2019-03-13 09:07:01,063 [20383] INFO     thoth.adviser.python.advise:118: Scored 100 stacks in total
2019-03-13 09:07:01,065 [20383] INFO     thoth.adviser.python.advise:125: Filling package digests to software stacks
2019-03-13 09:07:01,077 [20383] WARNING  thoth.adviser.python.helpers:110: No hashes found for package 'flask'

CPU times: user 5.27 s, sys: 282 ms, total: 5.55 s
Wall time: 8.74 s


Parameter `count` limits number of stacks provided in the output, parameter `limit` limits numbef of stacks scored in total.

The step above did not consider any runtime environment (user did not configure it in Thoth's configuration file). This means Thoth is assuming "any" runtime environment - this is also reported back to user as a recommendation.

In [15]:
stack_info

[{'type': 'WARNING',
  'justification': "Please specify Python version in Pipfile using `pipenv --python <VERSION>` and in Thoth's configuration file to have reproducible deployment and recommendations targeting specific Python version"}]

Let's check recommended changes in configuration by Thoth. As seen above, there is a warning message about Python version not being set in Thoth's configuration file. As there was no Python version stated in Pipfile, Thoth was not able to figure out what version it shoud advise:

In [16]:
advised_configuration is None

True

Also, take a look at the most important part - generated reports with stacks and guidenance/justifications:

In [17]:
report

[([{'score': 0.0}],
  Project(pipfile=Pipfile(packages=Packages(develop=False, packages={'flask': PackageVersion(name='flask', version='*', develop=False, index=None, hashes=[], markers=None, _semantic_version=None, _version_spec=None), 'pyyaml': PackageVersion(name='pyyaml', version='*', develop=False, index=None, hashes=[], markers=None, _semantic_version=None, _version_spec=None)}), dev_packages=Packages(develop=True, packages={}), meta=PipfileMeta(sources={'pypi-org': Source(url='https://pypi.org/simple', name='pypi-org', verify_ssl=True, warehouse=True, warehouse_api_url=None)}, requires={}, pipenv=None, hash=None, pipfile_spec=None)), pipfile_lock=PipfileLock(packages=Packages(develop=False, packages={'flask': PackageVersion(name='flask', version='==1.0.2', develop=False, index=Source(url='https://pypi.org/simple', name='pypi-org', verify_ssl=True, warehouse=True, warehouse_api_url=None), hashes=[], markers=None, _semantic_version=Version('1.0.2'), _version_spec=None), 'werkzeug'

In [18]:
# The best stack according to Thoth:
report[0][1].to_dict()

{'requirements': {'packages': {'flask': '*', 'pyyaml': '*'},
  'dev-packages': {},
  'source': [{'url': 'https://pypi.org/simple',
    'verify_ssl': True,
    'name': 'pypi-org'}]},
 'requirements_locked': {'_meta': {'sources': [{'url': 'https://pypi.org/simple',
     'verify_ssl': True,
     'name': 'pypi-org'}],
   'requires': {},
   'hash': {'sha256': 'e1114850139cb81c3f762470c4f7cd8023b173c336193601fc90dd711e3fb2de'},
   'pipfile-spec': 6},
  'default': {'flask': {'version': '==1.0.2',
    'hashes': [],
    'index': 'pypi-org'},
   'werkzeug': {'version': '==0.14.1', 'hashes': [], 'index': 'pypi-org'},
   'jinja2': {'version': '==2.10', 'hashes': [], 'index': 'pypi-org'},
   'itsdangerous': {'version': '==1.1.0', 'hashes': [], 'index': 'pypi-org'},
   'click': {'version': '==7.0', 'hashes': [], 'index': 'pypi-org'},
   'markupsafe': {'version': '==1.1.1', 'hashes': [], 'index': 'pypi-org'}},
  'develop': {}}}

Most of the tools out there expect there is a fix for a CVE in a more recent version of a package, which is obviously, not always true. As Thoth is a pro-active system (we use bots), when there is a new vulnerability, it can directly recommend to *downgrade* or *upgrade* version of a package based on observations Thoth stores.

The scenario above showed how Thoth works solely on package-level - we have considered how different packages affect "security" of a software stack (how many CVEs are present in a software stack). This way we have proved "package-level guidenance" for stacks. The previous scenario - scenario 2: `runtime-environment` showed guidenance on software stack level - when we have information how well a group of packages work together (performace related advises). The current implementation of Thoth combines these two input vectors - security guidenance and performance guidenance in the `STABLE` scoring function.

Feel free to experiment with parameters to adviser:

In [19]:
from thoth.adviser import RecommendationType

# Set recommendation type to one of the following:
[e.name for e in RecommendationType]

['STABLE', 'TESTING', 'LATEST']

In [20]:
# Adjust version ranges of Python packages being installed. These versions are resolved using
# pip's internal algorithm, so anything which is compatible with PEP-440 (and Pipfile compatible
# for Pipfile inputs) works out of box. Note this resolution is not done by installing
# dependencies (as in case of Pip/Pipenv), but there is implemented resolver on top of
# graph database which can resolve dependencies much faster as all the data are pre-computed.

PIPFILE_STR = """
[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
flask = "*"
pyyaml = "*"

# Allow pre-releases.
[pipenv]
allow_prereleases = true
"""

# In the example above, there is also used Pipfile.lock. Actually, Pipfile.lock is not used in
# recommendations, but Thoth stores it internally to track user's stacks, their evolution and
# changes to give better guidenance.
project = Project.from_strings(PIPFILE_STR)
project.to_dict()

# Mind dependencies resolved in solver run, unknown dependencies to Thoth, obviously, cannot be resolved by Thoth.

{'requirements': {'packages': {'flask': '*', 'pyyaml': '*'},
  'dev-packages': {},
  'source': [{'url': 'https://pypi.org/simple',
    'verify_ssl': True,
    'name': 'pypi'}],
  'pipenv': {'allow_prereleases': True}},
 'requirements_locked': None}

All of the above can be accomplished using Thamos CLI (as the above is more in-depth description what Thoth does on lower level). From user's perspective a user just installs `Thamos`, adjusts configuration via `thamos config` (automatic discovery of available hardware is performed) and issues `thamos advise` which talks to a Thoth deployment. All of the above is transparent to the user, the report is shown in a well formatted table. [Follow README instructions in thamos repo](https://github.com/thoth-station/thamos/tree/master/examples/scoring/) to experience this on your own.


Happy hacking! ;-)

In [21]:
from thoth.lab import packages_info

# Let's state Thoth's package versions for reproducible next runs of this Jupyter Notebook.
packages_info()

2019-03-13 09:07:02,463 [20383] INFO     root:126: Logging to a Sentry instance is turned off
2019-03-13 09:07:02,464 [20383] INFO     root:148: Logging to rsyslog endpoint is turned off


,package,version,importable
0,thoth.adviser,0.3.0,True
1,thoth.analyzer,0.1.2,True
2,thoth.common,0.7.1,True
3,thoth.lab,0.0.3,True
4,thoth.package_extract,1.0.1,True
5,thoth.python,0.4.6,True
6,thoth.solver,1.1.0,True
7,thoth.storages,0.9.6,True
8,thoth.worker,0.0.2,True
